## Seismic Data Analysis Notebook ##


In [ ]:
## Step 1: Load and Inspect Seismic Data
from obspy import read
import matplotlib.pyplot as plt
import numpy as np
from scipy.signal import spectrogram

# Load the seismic data (update the path to your data file)
st = read("path/to/your/datafile.mseed")

# Display the data summary
print(st)

# Plot the waveform
st.plot()


In [ ]:

## Step 2: Basic Signal Processing

# Copy the data and apply a bandpass filter (1-4 Hz, adjust as needed)
st_filtered = st.copy()
st_filtered.filter("bandpass", freqmin=1, freqmax=4)
st_filtered.plot()

# Plot raw and filtered waveforms
fig, ax = plt.subplots(2, 1, figsize=(10, 6))
ax[0].plot(st[0].times(), st[0].data, color="blue")
ax[0].set_title("Raw Waveform")
ax[1].plot(st_filtered[0].times(), st_filtered[0].data, color="orange")
ax[1].set_title("Filtered Waveform (1-4 Hz)")
plt.tight_layout()
plt.show()

## Step 3: Frequency Analysis with Short-Time Fourier Transform (STFT)

# Select a single trace if there are multiple in your stream
tr = st_filtered[0]
f, t, Sxx = spectrogram(tr.data, fs=tr.stats.sampling_rate, nperseg=1024)

# Plot the spectrogram
plt.figure(figsize=(12, 6))
plt.pcolormesh(t, f, 10 * np.log10(Sxx), shading='gouraud')
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [sec]')
plt.title("Spectrogram (1-hour Window)")
plt.colorbar(label="Intensity [dB]")
plt.show()

## Step 4: Detect Seismic Events (Peak Detection)
# Example STA/LTA (Short-Term Average / Long-Term Average) detection
from obspy.signal.trigger import classic_sta_lta, trigger_onset

# Apply STA/LTA algorithm
sta = 1   # short time window in seconds
lta = 10  # long time window in seconds
sampling_rate = tr.stats.sampling_rate
cft = classic_sta_lta(tr.data, int(sta * sampling_rate), int(lta * sampling_rate))

# Set thresholds for event detection
on_threshold = 2.5
off_threshold = 1.0
triggers = trigger_onset(cft, on_threshold, off_threshold)

# Plot results with detected events
fig, ax = plt.subplots(figsize=(12, 6))
ax.plot(tr.times(), tr.data, label="Waveform")
for trigger in triggers:
    ax.axvspan(tr.times()[trigger[0]], tr.times()[trigger[1]], color="red", alpha=0.3)
ax.set_xlabel("Time [s]")
ax.set_ylabel("Amplitude")
ax.set_title("Event Detection with STA/LTA")
plt.legend()
plt.show()

## Step 5: Basic Statistical Analysis of Amplitude

# Calculate basic statistics on the raw signal
amplitude_stats = {
    "mean": np.mean(tr.data),
    "median": np.median(tr.data),
    "std_dev": np.std(tr.data),
    "max_amplitude": np.max(tr.data)
}
print("Amplitude Statistics:", amplitude_stats)

## Step 6: Frequency Analysis (Fourier Transform and Power Spectral Density)

# Compute Fourier Transform
fft_values = np.fft.fft(tr.data)
fft_frequencies = np.fft.fftfreq(len(fft_values), d=1 / sampling_rate)

# Plot the power spectral density
plt.figure(figsize=(12, 6))
plt.plot(fft_frequencies[:len(fft_values) // 2], np.abs(fft_values[:len(fft_values) // 2])**2)
plt.xlabel("Frequency [Hz]")
plt.ylabel("Power Spectral Density")
plt.title("Power Spectral Density (Fourier Transform)")
plt.show()
